In [1]:
import pandas as pd
import numpy as np
import pickle
from preprocess_text import *
from bigrams import *
from smoothers import *
from LanguageModels import *

In [2]:
df = pd.read_csv('A1_dataset.csv')

In [3]:
# text = df['TEXT'].to_list()
# def preprocess_text(text):
#     text = lowercase_text(text)
#     text = remove_url_html(text)
#     text = remove_users(text)
#     text = remove_punctuations(text)
#     text = remove_whitespaces(text)
#     text = tokenization(text)
#     text = spelling_correction(text, 'autocorrect_full')
#     text = remove_alphanum(text)
#     return text
# df['preprocessed_text'] = df['TEXT'].progress_apply(preprocess_text)
# preprocess_text = df['preprocessed_text'].to_list()
# df.to_csv('preprocessed_A1.csv', index=False)

In [4]:
df = pd.read_csv('preprocessed_A1.csv')
preprocess_text = df['preprocessed_text'].to_list()
for i in range(len(preprocess_text)):
    preprocess_text[i] = eval(preprocess_text[i])
    

In [5]:
unigram_counts = {}
for sentence in preprocess_text:
    for word in sentence:
        if word in unigram_counts:
            unigram_counts[word] += 1
        else:
            unigram_counts[word] = 1

In [6]:
unigram_probs = {}
scale_factor = sum(unigram_counts.values())
for word in unigram_counts:
    unigram_probs[word] = unigram_counts[word] / scale_factor

In [7]:
len(unigram_counts)

6952

# Saving Bigrams with Smoothing

In [8]:
# bigram_wrapper = Bigrams(preprocess_text)
# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

# with open('bigram_model_dumps/bigram_wrapper.pickle', 'wb') as f:
#     pickle.dump(bigram_wrapper, f)

In [9]:
# load bigram_wrapper
# with open('bigram_wrapper.pickle', 'rb') as f:
#     bigram_wrapper = pickle.load(f)

# bigrams = bigram_wrapper.get_bigrams()
# vocab_len = bigram_wrapper.vocab_len

In [10]:
# smoother = LMSmoothers(bigrams, vocab_len, unigram_counts=unigram_counts, unigrams_probs=unigram_probs)
# laplace_smoothed_bigrams = smoother.laplace()
# add_5_smoothed_bigrams = smoother.add_k(5)
# add_5_with_unigram_prior_smoothed_bigrams = smoother.add_k_with_unigram_prior(5)

In [11]:
# with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(laplace_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'wb') as handle:
#     pickle.dump(add_5_with_unigram_prior_smoothed_bigrams, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Creating BiGram Models

In [12]:
with open('bigram_model_dumps/laplace_smoothed_bigrams.pickle', 'rb') as handle:
    laplace_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_smoothed_bigrams = pickle.load(handle)

# with open('bigram_model_dumps/add_5_with_unigram_prior_smoothed_bigrams.pickle', 'rb') as handle:
#     add_5_with_unigram_prior_smoothed_bigrams = pickle.load(handle)

In [13]:
# Load Negative Prompts
with open('Pos Neg Prompts/neg_250_hf.txt', 'rb') as handle:
    neg_prompts = handle.readlines()

for i in range(len(neg_prompts)):
    neg_prompts[i] = neg_prompts[i].decode('utf-8').strip()

In [14]:
# clear the file
open('neg_hf_prompts.txt', 'w').close()

In [15]:
def append_to_file(filename, text1, text2):
    with open(filename, 'a') as f:
        f.write(text1 + " " + text2 + '\n')

In [16]:
ls_neg_gen = []
lm = LanguageModel(laplace_smoothed_bigrams, unigram_probs, "vader", False, False, True, repetition_penalty=0.2, normalizebyPerplexity=True, sentiment_scale_factor=1)
for p in neg_prompts:
    sentence_neg = lm.generate_text([p], sentiment=-1, length=10)
    print("Negative: {}".format(sentence_neg))
    ppl = lm.computePerplexity(sentence_neg)
    print("Negative Perplexity: {}".format(ppl))
    ls_neg_gen.append((sentence_neg, ppl))
    append_to_file('neg_hf_prompts.txt', " ".join(sentence_neg), str(ppl))

Negative: ['pathetic', 'hell', 'everybody', 'see', 'you', 'raping', 'kill', 'myself', 'in', 'killed']
Negative Perplexity: 3703.6524377097257
Negative: ['lagging', 'hell', 'get', 'slavery', 'killed', 'the', 'worst', 'part', 'of', 'my']
Negative Perplexity: 2689.8469168530464
Negative: ['irritating', 'slavery', 'killed', 'the', 'shittiest', 'hell', 'killing', 'me', 'raping', 'horrific']
Negative Perplexity: 4837.780965025023
Negative: ['messes', 'slavery', 'hell', 'my', 'killed', 'tea', 'so', 'bad', 'raping', 'evil']
Negative Perplexity: 4447.7552576720245
Negative: ['appalling', 'kill', 'me', 'sad', 'killed', 'a', 'bad', 'hell', 'yeah', 'raping']
Negative Perplexity: 3666.5612733438256
Negative: ['worst', 'dreams', 'and', 'crisis', 'kill', 'slavery', 'raping', 'killing', 'mee', 'tragedy']
Negative Perplexity: 5116.034543991969
Negative: ['garbage', 'slavery', 'kill', 'raping', 'tragedy', 'arms', 'for', 'the', 'hell', 'yeah']
Negative Perplexity: 3007.7760066118663
Negative: ['useless',